In [1]:
import pandas as pd
from google.colab import drive
import os

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/JMU_Wu/NLP/eng_train.csv'
df = pd.read_csv(file_path)


def split_sentences(sentence):
    sentences = sentence.split('\n')
    first_sentence = sentences[0].strip()
    second_sentence = sentences[1].strip() if len(sentences) > 1 else ''
    return first_sentence, second_sentence

pair_id = df['PairID']
text = df['Text']
score = df['Score']

sentences = [split_sentences(sentence) for sentence in text]

supervised_data = pd.DataFrame({
    'PairID': pair_id,
    'FirstSentence': [sentence[0] for sentence in sentences],
    'SecondSentence': [sentence[1] for sentence in sentences],
    'Score': score
})

save_path = '/content/drive/MyDrive/JMU_Wu/NLP/'
supervised_data.to_csv(os.path.join(save_path, 'supervised_data.csv'), index=False)


Mounted at /content/drive


In [2]:
import pandas as pd
import os


supervised_data = pd.read_csv('/content/drive/MyDrive/JMU_Wu/NLP/supervised_data.csv')


cleaned_data = supervised_data.dropna(subset=['SecondSentence'])


save_path = '/content/drive/MyDrive/JMU_Wu/NLP/'
cleaned_data.to_csv(os.path.join(save_path, 'cleaned_data.csv'), index=False)


In [3]:
pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import re

cleaned_data = pd.read_csv('/content/drive/MyDrive/JMU_Wu/NLP/cleaned_data.csv')

def remove_punctuation(text):
    pattern = '[^\w\s]'
    text = re.sub(pattern, '', text)
    return text

cleaned_data['CleanedFirstSentence'] = cleaned_data['FirstSentence'].apply(remove_punctuation)
cleaned_data['CleanedSecondSentence'] = cleaned_data['SecondSentence'].apply(remove_punctuation)

cleaned_data.to_csv('/content/drive/MyDrive/JMU_Wu/NLP/cleaned_data_nopunc.csv', index=False)

In [5]:
pip install nltk

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
pip install --upgrade torch torchvision torchaudio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 926.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [8]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [9]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
import time
import warnings
import sys

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


from transformers import logging
logging.set_verbosity_error()

df = pd.read_csv('/content/drive/MyDrive/JMU_Wu/NLP/cleaned_data_nopunc.csv')

# Split - train and validation
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

class SemanticRelatednessDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pair = self.data.iloc[idx]
        first_sentence = pair['FirstSentence']
        second_sentence = pair['SecondSentence']
        relatedness_score = pair['Score']

        inputs = self.tokenizer(
            first_sentence,
            second_sentence,
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding='max_length'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(relatedness_score, dtype=torch.float32)
        }

# Create the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)

train_dataset = SemanticRelatednessDataset(train_df, tokenizer)
val_dataset = SemanticRelatednessDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 1

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total Training Time: {elapsed_time} seconds")

model_save_path = "semantic_relatedness_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Epoch 1/1 - Average Loss: 0.040911303380991165
Total Training Time: 80.27763104438782 seconds


('semantic_relatedness_model/tokenizer_config.json',
 'semantic_relatedness_model/special_tokens_map.json',
 'semantic_relatedness_model/vocab.json',
 'semantic_relatedness_model/merges.txt',
 'semantic_relatedness_model/added_tokens.json')

In [10]:
#Visual Inspection
import time

start_time = time.time()
end_time = 0
elapsed_time = 0
model.eval()
with torch.no_grad():
    for i, batch in enumerate(val_loader):
        # if i >= 5:
        #     break

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_scores = outputs.logits.squeeze().cpu().numpy()

        print(f"Case {i + 1}:")
        print("Predicted Score:", predicted_scores)
        print("Actual Score:", labels.cpu().numpy())
        print()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total Time taken: {elapsed_time} seconds")

Case 1:
Predicted Score: [0.28334534 0.7109908  0.29587695 0.2760692  0.49006388 0.3880727
 0.7041575  0.7661695  0.48562866 0.58617526 0.56384057 0.6953255
 0.7935943  0.7495378  0.29849306 0.4841586 ]
Actual Score: [0.25 0.69 0.16 0.28 0.59 0.44 0.91 0.97 0.41 0.38 0.75 0.69 0.84 0.75
 0.36 0.66]

Case 2:
Predicted Score: [0.291881   0.27422187 0.78745544 0.3322822  0.7028634  0.3477114
 0.46212605 0.68339485 0.27812883 0.7447244  0.2768141  0.4017502
 0.6530322  0.30514947 0.35508975 0.3606942 ]
Actual Score: [0.41 0.19 1.   0.56 0.62 0.56 0.44 0.81 0.43 0.81 0.12 0.25 0.75 0.25
 0.56 0.34]

Case 3:
Predicted Score: [0.72393405 0.71664846 0.72438323 0.74344695 0.64649236 0.5614458
 0.2927882  0.28097582 0.44898775 0.4580361  0.3694382  0.41587457
 0.57510245 0.6174827  0.4233035  0.7889886 ]
Actual Score: [0.5  0.81 0.59 0.72 0.84 0.53 0.31 0.28 0.44 0.59 0.28 0.12 0.1  0.47
 0.47 0.84]

Case 4:
Predicted Score: [0.5110687  0.57550347 0.5102768  0.42451447 0.7071067  0.35114673
 0.3

In [11]:
pip install scipy

In [12]:
from scipy.stats import pearsonr
import time

start_time = time.time()
end_time = 0
elapsed_time = 0
model.eval()
predicted_scores_all = []
labels_all = []

with torch.no_grad():
    for i, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_scores = outputs.logits.squeeze().cpu().numpy()

        # Accumulate predicted scores and labels
        predicted_scores_all.extend(predicted_scores)
        labels_all.extend(labels.cpu().numpy())

# Calculate Pearson correlation coefficient
correlation_coefficient, _ = pearsonr(predicted_scores_all, labels_all)
print("Pearson Correlation Coefficient:", correlation_coefficient)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total Time taken: {elapsed_time} seconds")

Pearson Correlation Coefficient: 0.805917429825787
Total Time taken: 13.233540058135986 seconds


In [13]:
from scipy.stats import spearmanr
import time

start_time = time.time()
end_time = 0
elapsed_time = 0
model.eval()
predicted_scores_all = []
labels_all = []

with torch.no_grad():
    for i, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_scores = outputs.logits.squeeze().cpu().numpy()

        # Accumulate predicted scores and labels
        predicted_scores_all.extend(predicted_scores)
        labels_all.extend(labels.cpu().numpy())

# Calculate Spearman correlation coefficient
correlation_coefficient, _ = spearmanr(predicted_scores_all, labels_all)
print("Spearman Correlation Coefficient:", correlation_coefficient)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total Time taken: {elapsed_time} seconds")

Spearman Correlation Coefficient: 0.8070217508640404
Total Time taken: 13.180181503295898 seconds


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
import time
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

import time

start_time = time.time()
end_time = 0
elapsed_time = 0

df = pd.read_csv('/content/drive/MyDrive/JMU_Wu/NLP/cleaned_data_nopunc.csv')

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

def create_dataset(dataframe, tokenizer, max_length=128):
    return SemanticRelatednessDataset(dataframe, tokenizer, max_length)

learning_rates = [5e-5, 3e-5, 2e-5]
epochs_to_run = [3, 5]
seeds = [42, 123, 456]

all_final_accuracies = []
all_epoch_losses = []
all_pearson_correlations = []
all_spearman_correlations = []

for learning_rate in learning_rates:
    for num_epochs in epochs_to_run:
        for seed in seeds:
            tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
            model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)

            train_dataset = create_dataset(train_df, tokenizer)
            val_dataset = create_dataset(val_df, tokenizer)

            train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model.to(device)
            optimizer = AdamW(model.parameters(), lr=learning_rate)
            num_epochs = num_epochs

            epoch_losses = []
            epoch_accuracies = []

            start_time = time.time()

            for epoch in range(num_epochs):
                model.train()
                total_loss = 0

                for batch in train_loader:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)

                    optimizer.zero_grad()

                    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    total_loss += loss.item()

                    loss.backward()
                    optimizer.step()

                average_loss = total_loss / len(train_loader)
                epoch_losses.append(average_loss)

                model.eval()
                total_correct = 0
                total_samples = 0

                with torch.no_grad():
                    for batch in val_loader:
                        input_ids = batch['input_ids'].to(device)
                        attention_mask = batch['attention_mask'].to(device)
                        labels = batch['labels'].to(device)

                        outputs = model(input_ids, attention_mask=attention_mask)
                        predicted_scores = outputs.logits.squeeze()

                        predicted_classes = torch.round(torch.sigmoid(predicted_scores))
                        correct_predictions = (predicted_classes == labels).sum().item()
                        total_correct += correct_predictions
                        total_samples += labels.size(0)

                accuracy = total_correct / total_samples
                epoch_accuracies.append(accuracy)

                print(f"Seed {seed} - Learning Rate {learning_rate} - Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss}")

            all_epoch_losses.append(epoch_losses)
            all_final_accuracies.append(epoch_accuracies)

            # Calculate Pearson correlation coefficient
            predicted_scores_all = []
            labels_all = []

            with torch.no_grad():
                for batch in val_loader:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)

                    outputs = model(input_ids, attention_mask=attention_mask)
                    predicted_scores = outputs.logits.squeeze().cpu().numpy()

                    # Accumulate predicted scores and labels
                    predicted_scores_all.extend(predicted_scores)
                    labels_all.extend(labels.cpu().numpy())

            correlation_coefficient, _ = pearsonr(predicted_scores_all, labels_all)
            all_pearson_correlations.append(correlation_coefficient)

            # Calculate Spearman correlation coefficient
            correlation_coefficient, _ = spearmanr(predicted_scores_all, labels_all)
            all_spearman_correlations.append(correlation_coefficient)

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Seed {seed} - Learning Rate {learning_rate} - Time Elapsed: {elapsed_time} seconds")

            model_save_path = f"semantic_relatedness_model_lr{learning_rate}_epochs{num_epochs}_seed{seed}"
            model.save_pretrained(model_save_path)
            tokenizer.save_pretrained(model_save_path)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total Time taken: {elapsed_time} seconds")

# Plotting
plt.figure(figsize=(14, 10))

# Loss Analysis
plt.subplot(2, 2, 1)
for i, epoch_losses in enumerate(all_epoch_losses):
    plt.plot(range(1, len(epoch_losses) + 1), epoch_losses, label=f"Run {i+1}")
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Training Loss vs. Epochs')
plt.legend()

# Accuracy Analysis
plt.subplot(2, 2, 2)
for i, epoch_accuracies in enumerate(all_final_accuracies):
    plt.plot(range(1, len(epoch_accuracies) + 1), epoch_accuracies, label=f"Run {i+1}")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Epochs')
plt.legend()

# Correlation Coefficient Analysis

plt.subplot(2, 2, 3)
plt.bar(range(len(all_pearson_correlations)), all_pearson_correlations, label='Pearson')
plt.xlabel('Run')
plt.ylabel('Correlation Coefficient')
plt.title('Pearson Correlation Coefficient')
plt.xticks(range(len(all_pearson_correlations)), labels=[f"Run {i+1}" for i in range(len(all_pearson_correlations))])

plt.subplot(2, 2, 4)
plt.bar(range(len(all_spearman_correlations)), all_spearman_correlations, label='Spearman')
plt.xlabel('Run')
plt.ylabel('Correlation Coefficient')
plt.title('Spearman Correlation Coefficient')
plt.xticks(range(len(all_spearman_correlations)), labels=[f"Run {i+1}" for i in range(len(all_spearman_correlations))])

plt.tight_layout()
plt.show()


Seed 42 - Learning Rate 5e-05 - Epoch 1/3 - Average Loss: 0.04141363921321251
Seed 42 - Learning Rate 5e-05 - Epoch 2/3 - Average Loss: 0.022952139378863978
Seed 42 - Learning Rate 5e-05 - Epoch 3/3 - Average Loss: 0.017297263951040805
Seed 42 - Learning Rate 5e-05 - Time Elapsed: 363.3653564453125 seconds
Seed 123 - Learning Rate 5e-05 - Epoch 1/3 - Average Loss: 0.06031945235857909
Seed 123 - Learning Rate 5e-05 - Epoch 2/3 - Average Loss: 0.05438456682826985
Seed 123 - Learning Rate 5e-05 - Epoch 3/3 - Average Loss: 0.05384037649766965
Seed 123 - Learning Rate 5e-05 - Time Elapsed: 360.23322916030884 seconds
Seed 456 - Learning Rate 5e-05 - Epoch 1/3 - Average Loss: 0.03744379310614684
Seed 456 - Learning Rate 5e-05 - Epoch 2/3 - Average Loss: 0.021824573401615702
Seed 456 - Learning Rate 5e-05 - Epoch 3/3 - Average Loss: 0.014946997844275426
Seed 456 - Learning Rate 5e-05 - Time Elapsed: 360.58009934425354 seconds
Seed 42 - Learning Rate 5e-05 - Epoch 1/5 - Average Loss: 0.03829449

In [ ]:
# # Print out the accuracies of every run
# for i in range(len(all_final_accuracies)):
#     print(f"Run {i+1} - Final Accuracy: {all_final_accuracies[i][-1]:.4f}")

# # Detailed summary in tabular format
# summary_df = pd.DataFrame(columns=['Run', 'Learning Rate', 'Epoch', 'Average Loss', 'Accuracy'])

# for i in range(len(all_final_accuracies)):
#     for epoch, (loss, acc) in enumerate(zip(all_epoch_losses[i], all_final_accuracies[i]), 1):
#         summary_df = summary_df.append({'Run': i+1, 'Learning Rate': learning_rates[i // (len(epochs_to_run) * len(seeds))],
#                                         'Epoch': epoch, 'Average Loss': loss, 'Accuracy': acc}, ignore_index=True)

# print(summary_df)


In [ ]:
# import matplotlib.pyplot as plt

# # Assuming you have stored the training losses from different runs in the list `training_losses`

# # Plot training loss vs. epochs for each training run
# plt.figure(figsize=(10, 6))
# for i, loss in enumerate(training_losses):
#     plt.plot(range(1, num_epochs + 1), loss, label=f"Run {i + 1}")
# plt.xlabel('Epochs')
# plt.ylabel('Training Loss')
# plt.title('Training Loss vs. Epochs')
# plt.legend()
# plt.show()


import matplotlib.pyplot as plt

# Dummy training losses for 3 runs, each with 5 epochs
training_losses = [
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.6, 0.5, 0.4, 0.3, 0.2],
    [0.7, 0.6, 0.5, 0.4, 0.3]
]

# Plot training loss vs. epochs for each training run
plt.figure(figsize=(10, 6))
for i, loss in enumerate(training_losses):
    plt.plot(range(1, 6), loss, label=f"Run {i + 1}")
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Training Loss vs. Epochs')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
from google.colab import drive
import os


file_path = '/content/drive/MyDrive/JMU_Wu/NLP/tel_dev_with_labels.csv'
df = pd.read_csv(file_path)


def split_sentences(sentence):
    sentences = sentence.split('\n')
    first_sentence = sentences[0].strip()
    second_sentence = sentences[1].strip() if len(sentences) > 1 else ''
    return first_sentence, second_sentence

pair_id = df['PairID']
text = df['Text']
score = df['Score']

sentences = [split_sentences(sentence) for sentence in text]

supervised_data = pd.DataFrame({
    'PairID': pair_id,
    'FirstSentence': [sentence[0] for sentence in sentences],
    'SecondSentence': [sentence[1] for sentence in sentences],
    'Score': score
})

save_path = '/content/drive/MyDrive/JMU_Wu/NLP/'
supervised_data.to_csv(os.path.join(save_path, 'tel_supervised_data.csv'), index=False)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import time

start_time = time.time()
end_time = 0
elapsed_time = 0

val_df = pd.read_csv('/content/drive/MyDrive/JMU_Wu/NLP/esp_supervised_data.csv')

class MultilingualDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pair = self.data.iloc[idx]
        first_sentence = pair['FirstSentence']
        second_sentence = pair['SecondSentence']

        # Tokenize sentences
        inputs = self.tokenizer(
            first_sentence,
            second_sentence,
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding='max_length'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(pair['Score'], dtype=torch.float32)
        }

model = RobertaForSequenceClassification.from_pretrained('semantic_relatedness_model')
tokenizer = RobertaTokenizer.from_pretrained('semantic_relatedness_model')

# dataset instance
val_dataset = MultilingualDataset(val_df, tokenizer)

val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

predicted_scores_list = []
labels_list = []

model.eval()

with torch.no_grad():
    for i, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_scores = outputs.logits.squeeze()

        predicted_scores_list.extend(predicted_scores.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())

spearman_corr, _ = spearmanr(labels_list, predicted_scores_list)
print(f"Spearman Correlation Coefficient: {spearman_corr}")

plt.figure(figsize=(10, 6))
plt.scatter(labels_list, predicted_scores_list, color='blue', alpha=0.5)
plt.xlabel('Actual Scores')
plt.ylabel('Predicted Scores')
plt.title('Predicted Scores vs. Actual Scores')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = {
    'Language': ['English', 'Telugu', 'Marathi', 'Arabic', 'Amharic', 'Hausa', 'Spanish', 'Kinyarwanda'],
    'Spearman Correlation Coefficient': [0.838, 0.51, 0.447, 0.43, 0.399, 0.42, 0.599, 0.207]
}

df = pd.DataFrame(data)

print(df)

plt.figure(figsize=(10, 6))
plt.bar(df['Language'], df['Spearman Correlation Coefficient'], color='skyblue')
plt.xlabel('Language')
plt.ylabel('Spearman Correlation Coefficient')
plt.title('Spearman Correlation Coefficients for Different Languages')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Spearman correlation coefficients obtained from my model
your_model_data = {
    'Language': ['English', 'Telugu', 'Marathi', 'Arabic', 'Amharic', 'Hausa', 'Spanish', 'Kinyarwanda'],
    'Your Model': [0.838, 0.51, 0.447, 0.43, 0.399, 0.42, 0.599, 0.207]
}

# Spearman correlation coefficients reported in the previous paper
previous_paper_data = {
    'Language': ['English', 'Telugu', 'Marathi', 'Arabic', 'Amharic', 'Hausa', 'Spanish', 'Kinyarwanda'],
    'Previous Paper': [0.60, 0.58, 0.60, 0.17, 0.57, 0.04, 0.69, 0.13]
}

df_your_model = pd.DataFrame(your_model_data)
df_previous_paper = pd.DataFrame(previous_paper_data)

merged_df = pd.merge(df_your_model, df_previous_paper, on='Language')

print("Comparison of Spearman Correlation Coefficients:")
print(merged_df)

plt.figure(figsize=(10, 6))
plt.plot(merged_df['Language'], merged_df['Your Model'], marker='o', label='Your Model', color='blue')
plt.plot(merged_df['Language'], merged_df['Previous Paper'], marker='o', label='Previous Paper', color='red')
plt.xlabel('Language')
plt.ylabel('Spearman Correlation Coefficient using XLMR')
plt.title('Comparison of Spearman Correlation Coefficients using XLMR')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
